# Skip notebook test
(this notebook is not executed as part of the RAPIDS cuGraph CI process.)

--- 

### Timing 
When looking at the overall workflow, NetworkX and cuGraph do things differently.  For example, NetworkX spends a lot of time creating the graph data structure.  cuGraph on the other hand does a lazy creation of the data structure when an algorithm is called.  

To further complicate the comparison problem, NetworkX does not always return the answer.  In some cases it returns a generator that is then called to get the data.  

This benchmark will measure time from an analyst perspective, how long does it take to create the graph and run an algorithm.  

__What is not timed__:  Reading the data</p>
__What is timed__:     (1) creating a Graph, (2) running the algorithm (3) run any generators


---

### Algorithms
|        Algorithm        |  Type         | Graph | DiGraph |   Notes
| ------------------------|---------------|------ | ------- |-------------
| Katz                    | Centrality    |   X   |         | 
| Betweenness Centrality  | Centrality    |   X   |         | Estimated, k = 100
| Louvain                 | Community     |   X   |         | Uses python-louvain for comparison
| Triangle Counting       | Community     |   X   |         |
| WCC                     | Components    |       |    X    | Nx requires directed and returns a generator  
| Core Number             | Core          |   X   |         |  
| PageRank                | Link Analysis |       |    X    |
| Jaccard                 | Similarity    |   X   |         |
| BFS                     | Traversal     |   X   |         | No depth limit 
| SSSP                    | Traversal     |   X   |         | 


### Test Data
Data is generated using a  Recursive MATrix (R-MAT) graph generation algorithm



### Notes
* Running Betweenness Centrality on the full graph is prohibited using NetworkX.  Anything over k=100 can explode runtime to days


Notebook Credits

    
| Author        |    Date    |  Update             | cuGraph Version |  Test Hardware         |
| --------------|------------|---------------------|-----------------|------------------------|
| Don Acosta    | 1/12/2023  | Created             | 23.02 nightly   | Tesla A6000, CUDA 11.5 |


## Import Modules

In [1]:
# system and other
import gc
import os
from time import perf_counter
import numpy as np
import math

# rapids
import cugraph
import cudf

# NetworkX libraries
import networkx as nx

# RMAT data generator
from cugraph.generators import rmat

In [2]:
try: 
    import community
except ModuleNotFoundError:
    os.system('pip install python-louvain')
    import community

### Define the test data

In [3]:
# Test Files
# set the data argument for full test or quick test

data_full = {
    'data_11th'   :  11, 
    'data_14th'   :  14,
    'data_17th'  :   16,
    'data_19th'  :   18
}

# for quick testing
data_quick = {
   'data_9th' : 9,
}


# TODO: Was set to quick for test
data = data_full


### Generate data
The data is generated once for each size.

In [4]:
# Data reader - the file format is MTX, so we will use the reader from SciPy
def generate_data(scale, edgefactor=16):
    _gdf = rmat(
        scale,
        (2 ** scale) * edgefactor,
        0.57,
        0.19,
        0.19,
        42,
        clip_and_flip=False,
        scramble_vertex_ids=True,
        create_using=None,  # return edgelist instead of Graph instance
        mg=False
        )
    print('Generating a dataframe of ' + str(len(_gdf)) + '...')
    return _gdf

## Create Graph functions
There are two types of graphs created:
* Directed Graphs - calls to create_xx_digraph
* Undirected Graphs - calls to create_xx_ugraph <- fully symmeterized 

In [5]:
# NetworkX
def create_nx_digraph(_df):
    _gnx = nx.from_pandas_edgelist(_df,
                                   source='src',
                                   target='dst',
                                   edge_attr=None,
                                   create_using=nx.DiGraph)
    return _gnx

def create_nx_ugraph(_df):
    _gnx = nx.from_pandas_edgelist(_df,
                                   source='src',
                                   target='dst',
                                   edge_attr=None,
                                   create_using=nx.Graph)
    return _gnx


# cuGraph
def create_cu_digraph(_df, transpose=False):
    _g = cugraph.Graph(directed=True)
    _g.from_cudf_edgelist(_df,
                          source='src',
                          destination='dst',
                          renumber=False,
                          store_transposed=transpose)
    return _g

def create_cu_ugraph(_df,transpose=False):
    _g = cugraph.Graph(directed=False)
    _g.from_cudf_edgelist(_df,
                          source='src',
                          destination='dst',
                          renumber=False,
                          store_transposed=transpose)
    return _g

## Algorithm Execution

### Katz

In [6]:
def nx_katz(_df, alpha):
    t1 = perf_counter()
    _G = create_nx_ugraph(_df)
    _ = nx.katz_centrality(_G, alpha)
    t2 = perf_counter() - t1
    return t2

def cu_katz(_df, alpha):
    t1 = perf_counter()
    _G = create_cu_ugraph(_df, transpose=True)
    _ = cugraph.katz_centrality(_G, alpha)
    t2 = perf_counter() - t1
    return t2

def cu_katz_nx(_df, alpha):
    t1 = perf_counter()
    _G = create_nx_ugraph(_df)
    _ = cugraph.katz_centrality(_G, alpha)
    t2 = perf_counter() - t1
    return t2

### Betweenness Centrality

In [7]:
def nx_bc(_df, _k):
    t1 = perf_counter()
    _G = create_nx_ugraph(_df)
    _ = nx.betweenness_centrality(_G, k=_k)
    t2 = perf_counter() - t1
    return t2

def cu_bc(_df, _k):
    t1 = perf_counter()
    _G = create_cu_ugraph(_df)
    _ = cugraph.betweenness_centrality(_G, k=_k)
    t2 = perf_counter() - t1
    return t2


### Louvain

In [8]:
def nx_louvain(_df):
    t1 = perf_counter()
    _G = create_nx_ugraph(_df)
    parts = community.best_partition(_G)
    
    # Calculating modularity scores for comparison 
    _ = community.modularity(parts, _G)  
    
    t2 = perf_counter() - t1
    return t2

def cu_louvain(_df):
    t1 = perf_counter()
    _G = create_cu_ugraph(_df)
    _,_ = cugraph.louvain(_G)
    t2 = perf_counter() - t1
    return t2


### Triangle Counting

In [9]:
def nx_tc(_df):
    t1 = perf_counter()
    _G = create_nx_ugraph(_df)
    nx_count = nx.triangles(_G)
    
    # To get the number of triangles, we would need to loop through the array and add up each count
    count = 0
    for key, value in nx_count.items():
        count = count + value    
    
    t2 = perf_counter() - t1
    return t2

def cu_tc(_df):
    t1 = perf_counter()
    _G = create_cu_ugraph(_df)
    _ = cugraph.triangle_count(_G)
    t2 = perf_counter() - t1
    return t2


### WCC

In [10]:
def nx_wcc(_df):
    t1 = perf_counter()
    _G = create_nx_digraph(_df)
    gen = nx.weakly_connected_components(_G)

    list_of_digraphs = []

    for subgraph in gen:
        list_of_digraphs.append(nx.subgraph(_G, subgraph))
    
    t2 = perf_counter() - t1
    return t2

def cu_wcc(_df):
    t1 = perf_counter()
    _G = create_cu_digraph(_df)
    if _G.is_directed():
        _G = _G.to_undirected()
    _ = cugraph.weakly_connected_components(_G)
    t2 = perf_counter() - t1
    return t2


### Core Number

In [11]:
def nx_core_num(_df):
    t1 = perf_counter()
    _G = create_nx_ugraph(_df)
    _G.remove_edges_from(nx.selfloop_edges(_G))
    nx_count = nx.core_number(_G)
    
    count = 0
    for key, value in nx_count.items():
        count = count + value
    
    t2 = perf_counter() - t1
    return t2

def cu_core_num(_df):
    t1 = perf_counter()
    _G = create_cu_ugraph(_df)
    _ = cugraph.core_number(_G)
    t2 = perf_counter() - t1
    return t2


### PageRank

In [12]:
def nx_pagerank(_df):
    t1 = perf_counter()
    _G = create_nx_digraph(_df)
    if _G.is_directed():
        _G = _G.to_undirected()
    _ = nx.pagerank(_G)
    t2 = perf_counter() - t1
    return t2 

def cu_pagerank(_df):
    t1 = perf_counter()
    _G = create_cu_digraph(_df, transpose=True)
    if _G.is_directed():
        _G = _G.to_undirected()
    _ = cugraph.pagerank(_G)
    t2 = perf_counter() - t1
    return t2


### Jaccard

In [13]:
def nx_jaccard(_df):
    t1 = perf_counter()
    _G = create_nx_ugraph(_df)
    nj = nx.jaccard_coefficient(_G)
    t2 = perf_counter() - t1
    return t2

def cu_jaccard(_df):
    t1 = perf_counter()
    _G = create_cu_ugraph(_df)
    _ = cugraph.jaccard_coefficient(_G)
    t2 = perf_counter() - t1
    return t2


### BFS

In [14]:
def nx_bfs(_df):
    seed = _df['src'].min()
    t1 = perf_counter()
    _G = create_nx_ugraph(_df)
    nb = nx.bfs_edges(_G, seed)
    nb_list = list(nb) # gen -> list
    t2 = perf_counter() - t1
    return t2

def cu_bfs(_df):
    seed = _df['src'].min()
    t1 = perf_counter()
    _G = create_cu_ugraph(_df)
    _ = cugraph.bfs(_G, seed)
    t2 = perf_counter() - t1
    return t2


### SSSP

In [15]:
def nx_sssp(_df):
    seed = _df['src'].min()
    t1 = perf_counter()
    _G = create_nx_ugraph(_df)
    _ = nx.shortest_path(_G, seed)
    t2 = perf_counter() - t1
    return t2

def cu_sssp(_df):
    seed = _df['src'].min()
    t1 = perf_counter()
    _G = create_cu_ugraph(_df)
    _ = cugraph.sssp(_G, seed)
    t2 = perf_counter() - t1
    return t2


---

# Benchmark

In [16]:
# number of datasets
num_datasets = len(data)

In [17]:
# arrays to capture performance gains
names = []
algos = []

# Two dimension data [file, perf]
time_algo_nx = []          # NetworkX
time_algo_cu = []          # cuGraph
perf = []

algos.append("   ")

i = 0
for k,v in data.items():
    time_algo_nx.append([])
    time_algo_cu.append([])
    perf.append([])
    
    # Saved the file Name
    names.append(k)

    # read data
    gdf = generate_data(v)
    pdf = gdf.to_pandas()
    print(f"\tdata in gdf {len(gdf)} and data in pandas {len(pdf)}")

    # prep
    tmp_g = create_cu_ugraph(gdf)
    deg = tmp_g.degree()
    deg_max = deg['degree'].max()

    alpha = 1 / deg_max
    num_nodes = tmp_g.number_of_vertices()
    
    del tmp_g
    del deg
    
    
    #----- Algorithm order is same as defined at top ----
    
    #-- Katz 
    print("\tKatz  ", end = '')
    if i == 0: 
        algos.append("Katz")

    print("n.", end='')
    tx = nx_katz(pdf, alpha)
    print("c.", end='')
    tc = cu_katz(gdf, alpha)
    print("")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)

    gc.collect()
    
    
    #-- BC
    print("\tBC k=100  ", end='')
    if i == 0:
        algos.append("BC Estimate fixed")

    k = 100
    if k > num_nodes:
        k = int(num_nodes)
    print("n.", end='')
    tx = nx_bc(pdf, k)
    print("c.", end='')
    tc = cu_bc(gdf, k)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()
    

    #-- Louvain
    print("\tLouvain  ", end='')
    if i == 0:
        algos.append("Louvain")

    print("n.", end='')
    tx = nx_louvain(pdf)
    print("c.", end='')
    tc = cu_louvain(gdf)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()
    
    #-- TC
    print("\tTC  ", end='')
    if i == 0:
        algos.append("TC")

    print("n.", end='')
    tx = nx_tc(pdf)
    print("c.", end='')
    tc = cu_tc(gdf)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()


    #-- WCC
    print("\tWCC  ", end='')
    if i == 0:
        algos.append("WCC")

    print("n.", end='')
    tx = nx_wcc(pdf)
    print("c.", end='')
    tc = cu_wcc(gdf)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()
    
    #-- Core Number
    print("\tCore Number  ", end='')
    if i == 0:
        algos.append("Core Number")

    print("n.", end='')
    tx = nx_core_num(pdf)
    print("c.", end='')
    tc = cu_core_num(gdf)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()

    
    #-- PageRank
    print("\tPageRank  ", end='')
    if i == 0:
        algos.append("PageRank")

    print("n.", end='')
    tx = nx_pagerank(pdf)
    print("c.", end='')
    tc = cu_pagerank(gdf)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()
    
    
    #-- Jaccard
    print("\tJaccard  ", end='')
    if i == 0:
        algos.append("Jaccard")

    print("n.", end='')
    tx = nx_jaccard(pdf)
    print("c.", end='')
    tc = cu_jaccard(gdf)
    print(" ")
    
    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()
    

    #-- BFS
    print("\tBFS  ", end='')
    if i == 0:
        algos.append("BFS")

    print("n.", end='')
    tx = nx_bfs(pdf)
    print("c.", end='')
    tc = cu_bfs(gdf)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()
    
    
    #-- SSSP
    print("\tSSSP  ", end='')
    if i == 0:
        algos.append("SSP")

    print("n.", end='')
    tx = nx_sssp(pdf)
    print("c.", end='')
    tc = cu_sssp(gdf)
    print(" ")

    time_algo_nx[i].append(tx)
    time_algo_cu[i].append(tc)
    perf[i].append(tx/tc)
    gc.collect()

    # increament count
    
    i = i + 1

Generating a dataframe of 32768...
	data in gdf 32768 and data in pandas 32768
	Katz  n.c.
	BC k=100  n.c. 
	Louvain  n.c. 
	TC  n.c. 
	WCC  n.c. 
	Core Number  n.c. 
	PageRank  n.c.

/home/dacosta/dev_1230/cugraph/python/cugraph/cugraph/link_analysis/pagerank.py:180: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


 
	Jaccard  n.c. 
	BFS  n.c. 
	SSSP  n.c. 
Generating a dataframe of 262144...
	data in gdf 262144 and data in pandas 262144
	Katz  n.c.
	BC k=100  n.c. 
	Louvain  n.c. 
	TC  n.c. 
	WCC  n.c. 
	Core Number  n.c. 
	PageRank  n.c. 
	Jaccard  n.c. 
	BFS  n.c. 
	SSSP  n.c. 
Generating a dataframe of 2097152...
	data in gdf 2097152 and data in pandas 2097152
	Katz  n.c.
	BC k=100  n.

KeyboardInterrupt: 

In [ ]:
#Print results
print(algos)

for i in range(num_datasets):
    print(f"{names[i]}")
    print(f"{perf[i]}")

___
Copyright (c) 2020-2023, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___